In [ ]:
# Import other useful Python packages
import numpy as np
import matplotlib.pyplot as plt
import spikesorting_fullpursuit as fbp
from spikesorting_fullpursuit.parallel.spikesorting_parallel import spike_sort_parallel

from scipy.signal import butter, filtfilt

def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

import pickle

In [ ]:
channel_num = 32

data_dir = 'C:/Users/wanglab/Data/Licking/dirt/100523_3/'
filename = data_dir + 'amplifier.dat'

save_fname = data_dir + 'sorted_data.pickle'

neurons_fname = data_dir + 'sorted_neurons.pickle'

log_dir = data_dir + 'logs'

# Setup the sorting parameters dictionary.
spike_sort_args = {
        'sigma': 4.0, # Threshold based on noise level
        'clip_width': [-10e-4, 10e-4], # Width of clip in seconds
        'p_value_cut_thresh': 0.01,
        'segment_duration': np.inf, #None Uses entire recording
        'segment_overlap': 150,
        'do_branch_PCA': True,
        'do_branch_PCA_by_chan': True,
        'do_overlap_recheck': True,
        'filter_band': (300, 5000), #Intan filter bandwidth
        'do_ZCA_transform': True,
        'check_components': 20,
        'max_components': 5,
        'min_firing_rate': 0.1,
        'use_rand_init': True,
        'add_peak_valley': False,
        'max_gpu_memory': .1 * (1024 * 1024 * 1024),
        'save_1_cpu': True,
        'sort_peak_clips_only': True,
        'n_cov_samples': 100000, #Used to produce noise covariance matrix. Seems to plateau at 100,000
        'sigma_bp_noise': 2.326,
  #      'sigma_bp_CI': 12.0,
        'absolute_refractory_period': 10e-4,
        'get_adjusted_clips': False,
        'max_binary_pursuit_clip_width_factor': 1.0,
        'verbose': True,
        'test_flag': False,
        'log_dir': log_dir,
        }

In [ ]:
voltage = np.fromfile(filename, dtype=np.int16)
voltage = voltage.reshape(int(voltage.shape[0] / channel_num),channel_num)
voltage = voltage.transpose();
voltage = np.float32(voltage)

common_average =  np.median(voltage,0)

for channel in range(channel_num):
    print('Filtering channel %d' % channel)
    voltage[channel] = butter_bandpass_filter(
            voltage[channel] - common_average,
            300,
            5000,
            30000,
            order=4)

In [ ]:
""" xy_layout is 2D numpy array where each row represents its
corresonding channel number and each column gives the x, y coordinates
of that channel in micrometers. """

xy_layout = np.array([
    [21.65, 25], #0
    [21.65, 725], #1
    [21.65, 125], #2
    [21.65, 325], #3
    [21.65, 375], #4
    [21.65, 475], #5
    [21.65, 675], #6
    [21.65, 775], #7
    [21.65, 225], #8
    [21.65, 275], #9
    [21.65, 575], #10
    [21.65, 525], #11
    [21.65, 425], #12
    [21.65, 175], #13
    [21.65, 625], #14
    [21.65, 75], #15
    [-21.65, 0], #16
    [-21.65, 450], #17
    [-21.65, 100], #18
    [-21.65, 250], #19
    [-21.65, 600], #20
    [-21.65, 400], #21
    [-21.65, 550], #22
    [-21.65, 750], #23
    [-21.65, 200], #24
    [-21.65, 650], #25
    [-21.65, 350], #26
    [-21.65, 500], #27
    [-21.65, 700], #28
    [-21.65, 150], #29
    [-21.65, 300], #30
    [-21.65, 50] #31   
    ])

sorted_height = np.flip(xy_layout[:,1].argsort())

sampling_rate = 30000
neighbor_distance = 100 #Distance in microns to add channels to look for templates

#Let's arrange the channels spatially and the corresponding voltages. This will help for later manual analysis.
#Note that from this point on, Channel 1 will correspond to the "top" channel rather than Intan

xy_layout = xy_layout[sorted_height,:]
voltage = voltage[sorted_height,:]

In [ ]:
Probe = fbp.electrode.DistanceBasedProbe(sampling_rate,channel_num,xy_layout,100,voltage_array=voltage)

In [ ]:
#Probe.bandpass_filter_parallel(spike_sort_args['filter_band'][0], spike_sort_args['filter_band'][1])

In [ ]:
print("Start sorting")
sort_data, work_items, sorter_info = spike_sort_parallel(Probe, **spike_sort_args)

In [ ]:
print("Saving neurons file as", save_fname)
with open(save_fname, 'wb') as fp:
    pickle.dump((sort_data, work_items, sorter_info), fp, protocol=-1)

### Post Processing

In [ ]:
from spikesorting_fullpursuit.postprocessing import WorkItemSummary

In [ ]:
# First step in automated post-processing
# Set a few variables that can allow easy detection of units that are poor
absolute_refractory_period = 10e-4 # Refractory period (in ms) will be used to determine potential violations in sorting accuracy
# Max allowable ratio between refractory period violations and maximal bin of ACG. Units that violate will be deleted. Setting to >= 1. allows all units
max_mua_ratio = 1.
min_snr = 0 # Minimal SNR a unit must have to be included in post-processing
min_overlapping_spikes = .75 # Percentage of spikes required with nearly identical spike times in adjacent segments for them to combine in stitching

# Create the work_summary postprocessing object
work_summary = WorkItemSummary(sort_data, work_items,
                sorter_info, absolute_refractory_period=absolute_refractory_period,
                max_mua_ratio=max_mua_ratio, min_snr=min_snr,
                min_overlapping_spikes=min_overlapping_spikes, verbose=False)

# No segments in the demo (segment_duration > duration of synthetic data) but done as example
work_summary.stitch_segments()

# Summarize the sorted output data into dictionaries by time segment.
work_summary.summarize_neurons_by_seg()

# Finally summarize neurons across channels (combining and removing duplicate
# neurons across space) to get a list of sorted "neurons"
neurons = work_summary.summarize_neurons_across_channels(
                    overlap_ratio_threshold=np.inf, 
                    min_segs_per_unit=1,
                    remove_clips=False)

print("Saving neurons file as", neurons_fname)
with open(neurons_fname, 'wb') as fp:
    pickle.dump(neurons, fp, protocol=-1)



In [ ]:
print("Found", len(neurons), "total units with properties:")
fmtL = "Unit: {:.0f} on chans {}; n spikes = {:.0f}; FR = {:.0f}; Dur = {:.0f}; SNR = {:.2f}; MUA = {:.2f}; TolInds = {:.0f}"
for ind, n in enumerate(neurons):
    print_vals = [ind, n['channel'], n['spike_indices'].size, n['firing_rate'], n['duration_s'], n['snr']['average'], n['fraction_mua'], n['duplicate_tol_inds']]
    print(fmtL.format(*print_vals))


In [ ]:
from spikesorting_fullpursuit.utils import convert_to_viz
convert_to_viz.f_neurons_to_viz(neurons_fname,neuroviz_only=True)

### Plot Data

In [ ]:
start_win = round(0 * 30000)
end_win = round(250 * 30000)
clip_win = 30
offset = 3500

%matplotlib qt

coi = [30,31]
#coi = range(0,channel_num)

noi = [30,31]
#noi = range(0,len(neurons))

#We can sort the voltage array spatially before we plot

for n in coi:
    plt.plot(voltage[n,start_win:end_win] + n * offset,linewidth = 0.5, color="blue")

for n in range(0,len(neurons)):
    
    color = tuple(np.random.random(size=3))
    
    for spike in neurons[n]['spike_indices']:
        if ((spike > start_win) & (spike < end_win)):
            
            channels = Probe.get_neighbors(neurons[n]['channel'][0])
            
            for c in channels:
                voltage_range = range((spike-clip_win),(spike+clip_win))
                plot_range = range((spike-clip_win) - start_win,(spike+clip_win) - start_win)
                
                
                #plt.plot(plot_range,voltage[c,voltage_range] + c * offset,color=color, linewidth = 0.5)

In [ ]:
start_win = round(101.55 * 30000)
end_win = round(101.55 * 30000) + 1000
clip_win = 30

noi = 25;

plt.plot(voltage[noi,start_win:end_win],linewidth = 0.5, color="blue")

#for n in range(0,len(neurons)):
for n in range(32,34):
    
    channels = Probe.get_neighbors(neurons[n]['channel'][0])
    
    if any(x == noi for x in channels):
    
        color = tuple(np.random.random(size=3))
    
        for spike in neurons[n]['spike_indices']:
            if ((spike > start_win) & (spike < end_win)):
                voltage_range = range((spike-clip_win),(spike+clip_win))
                plot_range = range((spike-clip_win) - start_win,(spike+clip_win) - start_win)
                plt.plot(plot_range,voltage[noi,voltage_range] + n * 1000,color=color, linewidth = 1)

In [ ]:
# ACF from one unit
plt.hist(np.diff(neurons[32]['spike_indices']),bins=np.arange(0,3000,30));

In [ ]:
# ACF from another unit
plt.hist(np.diff(neurons[33]['spike_indices']),bins=np.arange(0,3000,30))

### Load Data

In [ ]:
with open(save_fname, 'rb') as fp:
    sorted_data = pickle.load(fp)

sort_data, work_items, sorter_info = sorted_data[0], sorted_data[1], sorted_data[2]

### Save voltage arranged by channel as dat file

In [ ]:
voltage_rearrange_filename = data_dir + 'voltage_sorted.dat'
voltage.astype('int16').tofile(voltage_rearrange_filename)